In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup
)
from torch.optim import AdamW


# =========================================================
# 0) 설정 (너 환경에 맞게 여기만 바꾸면 됨)
# =========================================================
MODEL_NAME = "kakaobank/kf-deberta-base"

LABELED_CSV = "/content/drive/MyDrive/labelling_hope.csv"     # ✅ -1/0/1만 있는 학습용 라벨 csv
PREDICT_CSV = "./to_predict.csv"        # ✅ 예측할 csv (없으면 학습 파일로 예시 예측)
OUT_PRED_CSV = "/content/drive/MyDrive/predicted_with_neutral.csv"

SAVE_DIR = "/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_hope"

TITLE_COL = "제목"
BODY_COL  = "본문"
LABEL_COL = "label"  # ✅ 라벨 컬럼명 (-1 또는 1)

MAX_LEN = 256        # 제목+본문이면 256 권장 (제목만이면 64)
EPOCHS = 5
LR = 2e-5
TRAIN_BS = 8         # DeBERTa 무거움: 8~16 추천
VAL_BS = 16
SEED = 42

# ✅ 중립(0) 임계값: 예측 때만 사용
TAU_CONF = 0.60      # max(prob) < 0.60 -> 중립
DELTA_MARGIN = 0.20  # |p_pos - p_neg| < 0.20 -> 중립


# =========================================================
# 1) 유틸
# =========================================================
def set_seed(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def softmax_np(x):
    x = x - np.max(x, axis=-1, keepdims=True)
    e = np.exp(x)
    return e / np.sum(e, axis=-1, keepdims=True)

set_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


# =========================================================
# 2) 라벨 데이터 로드
# =========================================================
df = pd.read_csv(LABELED_CSV, encoding="utf-8-sig")

if TITLE_COL not in df.columns:
    raise KeyError(f"'{TITLE_COL}' 컬럼이 없습니다. 현재 컬럼: {list(df.columns)}")

if BODY_COL not in df.columns:
    df[BODY_COL] = ""

if LABEL_COL not in df.columns:
    raise KeyError(f"'{LABEL_COL}' 컬럼이 없습니다. 현재 컬럼: {list(df.columns)}")

# 라벨 정리 (문자열/1.0/-1.0 방지)
df[LABEL_COL] = (
    df[LABEL_COL].astype(str).str.strip().str.replace(r"\.0$", "", regex=True)
)
df[LABEL_COL] = pd.to_numeric(df[LABEL_COL], errors="coerce")

# -1, 1만 남기기
df = df[df[LABEL_COL].isin([-1, 0, 1])].copy()

# ✅ 2클래스 매핑 (이게 네가 원한 "매핑 코드") -> Updated for 3-class indices
label_map = {-1: 0, 0 : 1, 1: 2}        # NEG=0, NEUTRAL=1, POS=2
inv_label_map = {0: -1, 1: 0, 2: 1}    # 예측값을 -1/0/1로 되돌릴 때

df["label_id"] = df[LABEL_COL].map(label_map).astype(int)

print("label 분포:\n", df[LABEL_COL].value_counts())
print("label_id 분포:\n", df["label_id"].value_counts())


# 입력 텍스트 (제목+본문)
df["text"] = df[TITLE_COL].astype(str) + "\n\n" + df[BODY_COL].astype(str)

train_df, val_df = train_test_split(
    df, test_size=0.2, random_state=SEED, stratify=df["label_id"]
)
print("train:", len(train_df), "val:", len(val_df))


# =========================================================
# 3) Dataset / DataLoader
# =========================================================
class NewsDataset(Dataset):
    def __init__(self, df_):
        self.texts = df_["text"].astype(str).tolist()
        self.labels = df_["label_id"].astype(int).tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = tokenizer(
            self.texts[idx],
            padding="max_length",
            truncation=True,
            max_length=MAX_LEN,
            return_tensors="pt"
        )
        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_loader = DataLoader(NewsDataset(train_df), batch_size=TRAIN_BS, shuffle=True)
val_loader   = DataLoader(NewsDataset(val_df), batch_size=VAL_BS, shuffle=False)


# =========================================================
# 4) 모델 (✅ 카카오뱅크 모델 기반 3클래스)
# =========================================================
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3
).to(device)

optimizer = AdamW(model.parameters(), lr=LR)

total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)


# =========================================================
# 5) 학습 / 평가
# =========================================================
def train_one_epoch(epoch):
    model.train()
    total_loss = 0.0
    pbar = tqdm(train_loader, desc=f"[TRAIN] Epoch {epoch+1}/{EPOCHS}")

    for batch in pbar:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        pbar.set_postfix(loss=total_loss / max(1, len(pbar)))

    return total_loss / max(1, len(train_loader))

def evaluate():
    model.eval()
    y_true, y_pred = [], []
    total_loss = 0.0

    with torch.no_grad():
        for batch in val_loader:
            labels = batch["labels"].cpu().numpy()
            batch = {k: v.to(device) for k, v in batch.items()}

            outputs = model(**batch)
            loss = outputs.loss
            logits = outputs.logits

            total_loss += loss.item()
            preds = torch.argmax(logits, dim=-1).cpu().numpy()

            y_true.extend(labels.tolist())
            y_pred.extend(preds.tolist())

    print("[VAL] loss:", total_loss / max(1, len(val_loader)))
    # Updated target_names for 3 classes (0, 1, 2) mapped to (-1, 0, 1)
    print(classification_report(y_true, y_pred, target_names=["NEG(-1)", "NEU(0)", "POS(1)"]))

for epoch in range(EPOCHS):
    tr_loss = train_one_epoch(epoch)
    print("train loss:", tr_loss)
    evaluate()


# =========================================================
# 6) 저장
# =========================================================
os.makedirs(SAVE_DIR, exist_ok=True)
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)
print("✅ 파인튜닝 모델 저장 완료:", SAVE_DIR)

device: cuda
label 분포:
 label
-1    5000
 1    2000
 0     500
Name: count, dtype: int64
label_id 분포:
 label_id
0    5000
2    2000
1     500
Name: count, dtype: int64
train: 6000 val: 1500


pytorch_model.bin:   0%|          | 0.00/746M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at kakaobank/kf-deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[TRAIN] Epoch 1/5: 100%|██████████| 750/750 [03:34<00:00,  3.49it/s, loss=0.367]


train loss: 0.36693786644376813
[VAL] loss: 0.1256656483053527
              precision    recall  f1-score   support

     NEG(-1)       0.98      0.98      0.98      1000
      NEU(0)       0.72      0.85      0.78       100
      POS(1)       0.98      0.94      0.96       400

    accuracy                           0.96      1500
   macro avg       0.89      0.92      0.91      1500
weighted avg       0.96      0.96      0.96      1500



[TRAIN] Epoch 2/5: 100%|██████████| 750/750 [03:35<00:00,  3.47it/s, loss=0.103]


...

train loss: 0.057330354689154775
[VAL] loss: 0.08621371812261443
              precision    recall  f1-score   support

     NEG(-1)       0.98      0.98      0.98      1000
      NEU(0)       0.83      0.89      0.86       100
      POS(1)       0.99      0.97      0.98       400

    accuracy                           0.97      1500
   macro avg       0.94      0.95      0.94      1500
weighted avg       0.98      0.97      0.97      1500



[TRAIN] Epoch 4/5: 100%|██████████| 750/750 [03:35<00:00,  3.47it/s, loss=0.0307]


train loss: 0.03067219253901082
[VAL] loss: 0.07111884156202382
              precision    recall  f1-score   support

     NEG(-1)       0.99      0.99      0.99      1000
      NEU(0)       0.86      0.82      0.84       100
      POS(1)       0.98      0.98      0.98       400

    accuracy                           0.98      1500
   macro avg       0.94      0.93      0.94      1500
weighted avg       0.98      0.98      0.98      1500



[TRAIN] Epoch 5/5: 100%|██████████| 750/750 [03:35<00:00,  3.47it/s, loss=0.0137]


train loss: 0.013702849656847926
[VAL] loss: 0.07116555442592065
              precision    recall  f1-score   support

     NEG(-1)       0.99      0.99      0.99      1000
      NEU(0)       0.87      0.87      0.87       100
      POS(1)       0.99      0.98      0.99       400

    accuracy                           0.98      1500
   macro avg       0.95      0.95      0.95      1500
weighted avg       0.98      0.98      0.98      1500

✅ 파인튜닝 모델 저장 완료: /content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_hope


In [ ]:
import os

In [ ]:
company_list = os.listdir("/content/drive/MyDrive/news_clean/")

In [ ]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification


for company in range()
  # ==========================================
  # 1. 설정 (학습 때와 동일하게 맞춰주세요)
  # ==========================================
  MODEL_PATH = "/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_hope" # 저장된 경로
  INPUT_CSV = "/content/drive/MyDrive/news_clean/KB금융_clean.csv"  # 예측하고 싶은 파일
  OUTPUT_CSV = "/content/drive/MyDrive/뉴스분류/KB금융_predicted_results_hope.csv" # 결과 저장 경로

  MAX_LEN = 256
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  # ==========================================
  # 2. 모델 및 토크나이저 로드
  # ==========================================
  print("모델 로딩 중...")
  tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
  model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
  model.eval()

  # ==========================================
  # 3. 데이터 로드 및 전처리
  # ==========================================
  df = pd.read_csv(INPUT_CSV, encoding="utf-8-sig")

  # 제목과 본문 합치기 (학습 때와 동일한 포맷)
  # 만약 본문 컬럼이 없다면 제목만 사용하도록 처리
  if "본문" not in df.columns:
      df["본문"] = ""

  texts = (df["제목"].astype(str) + "\n\n" + df["본문"].astype(str)).tolist()

  # ==========================================
  # 4. 예측 (Inference)
  # ==========================================
  results = []
  inv_label_map = {0: -1, 1: 0, 2: 1} # 0=부정, 1=중립, 2=긍정

  print(f"{len(texts)}개의 기사 분석 시작...")

  with torch.no_grad():
      for text in tqdm(texts):
          # 토큰화
          inputs = tokenizer(
              text,
              return_tensors="pt",
              truncation=True,
              max_length=MAX_LEN,
              padding="max_length"
          ).to(device)

          # 모델 추론
          outputs = model(**inputs)
          logits = outputs.logits

          # 확률값 계산 (Softmax)
          probs = torch.nn.functional.softmax(logits, dim=-1).cpu().numpy()[0]

          # 가장 높은 확률의 인덱스 선택
          pred_id = np.argmax(probs)
          pred_label = inv_label_map[pred_id]

          # 결과 저장 (확률값도 함께 저장하면 분석에 용이함)
          results.append({
              "pred_label": pred_label,
              "prob_neg": round(probs[0], 4),
              "prob_neu": round(probs[1], 4),
              "prob_pos": round(probs[2], 4)
          })

  # ==========================================
  # 5. 결과 합치기 및 저장
  # ==========================================
  pred_df = pd.DataFrame(results)
  final_df = pd.concat([df, pred_df], axis=1)

  # 감성을 한글로 보고 싶다면 추가
  final_df['sentiment'] = final_df['pred_label'].map({-1: "악재(부정)", 0: "중립", 1: "호재(긍정)"})

  final_df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
  print(f"✅ 예측 완료! 결과 저장됨: {OUTPUT_CSV}")

SyntaxError: expected ':' (ipython-input-4120088791.py, line 8)

In [ ]:
import pandas as pd

In [ ]:
df_pred_hope = pd.read_csv('/content/drive/MyDrive/대웅제약_predicted_results_hope.csv')
df_pred_hope

,일자,제목,본문,ticker,pred_label,prob_neg,prob_neu,prob_pos,sentiment
0,20150101,"대웅제약, “국민의 건강은 우리가” 신약개발에 밤낮 매진",◆ 광복 70돌 다시 뛰는 코리아 / 해방둥이 기업들의 도전 ◆\n 대웅제약은 19...,69620,1,0.0060,0.0029,0.9910,호재(긍정)
1,20150102,"대웅제약, 제약업계 최초 CP 등급평가 ‘AA’ 등급 획득",[이투데이] 강인효 기자(kangih@etoday.co.kr)대웅제약은 공정거래위원...,69620,0,0.0928,0.9053,0.0019,중립
2,20150102,"대웅제약, 2015년 시무식에서 ‘글로벌 제약기업 도약’ 다짐",[이투데이] 강인효 기자(kangih@etoday.co.kr)대웅제약이 글로벌 제약...,69620,0,0.0045,0.9941,0.0015,중립
3,20150104,"대웅제약 ""2015년 글로벌 제약기업 도약""",지속적인 R&D 투자와 글로벌 시장 거점 확대 통한 수출 활성화 목표[아시아경제 이...,69620,0,0.0105,0.9848,0.0047,중립
4,20150105,"대웅제약, CP등급평가 ‘AA’ 최고등급 획득",브레이크뉴스 문흥수 기자= 대웅제약이 공정거래위원회가 주관하는 CP 등급...,69620,0,0.0155,0.9828,0.0018,중립
...,...,...,...,...,...,...,...,...,...
7289,20241223,"대웅제약, 펙수클루 브라질 수출 공급계약 해지 “다른 파트너사 찾는다”",위식도역류질환 치료제 ‘펙수클루’. 대웅제약 \n대웅제약이 브라질 EMS S/A와 ...,69620,1,0.3262,0.3181,0.3557,호재(긍정)
7290,20241223,"대웅제약, 859억 규모 펙스클루 공급계약 해지...""브라질 EMS S/A 계약불이행""",[이데일리 김지완 기자] 대웅제약(069620)이 859억원 규모의 펙스클루 공급계...,69620,-1,0.9940,0.0049,0.0011,악재(부정)
7291,20241223,"대웅제약, 859억 규모 펙스클루 브라질 수출 공급계약 해지",대웅제약은 브라질 EMS S/A와 체결한 859억원 규모 펙수클루(성분명 펙수프라잔...,69620,1,0.2964,0.2574,0.4463,호재(긍정)
7292,20241223,"[공시]대웅제약, 펙수프라잔 브라질 수출 공급 계약 해지",[파이낸셜뉴스] 대웅제약은 지난 2020년 8월 체결한 펙수프라잔(Fexupraza...,69620,-1,0.9830,0.0114,0.0056,악재(부정)


In [ ]:
df_pred_hope['sentiment'].value_counts()

,count
sentiment,
중립,3265
호재(긍정),2785
악재(부정),1244


In [ ]:
labelling = pd.read_csv('/content/drive/MyDrive/labeling_60만.csv', dtype={'ticker':str})

In [ ]:
labelling.value_counts('label')

,count
label,
0,407201
1,151118
-1,49122


In [ ]:
label_1 = labelling.loc[labelling['label'] == 1]

In [ ]:
label_1.columns

Index(['Unnamed: 0', '일자', '제목', '본문', 'ticker', 'label'], dtype='object')

In [ ]:
label_1.drop(columns = 'Unnamed: 0',inplace=True)

/tmp/ipython-input-1115418367.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_1.drop(columns = 'Unnamed: 0', inplace=True)


In [ ]:
label_1.reset_index(drop=True, inplace=True)

In [ ]:
label_1.loc[13000:13050, ]

,일자,제목,본문,ticker,label
13000,2023-08-01,"KG모빌리티, 7월 1만848대 판매 수출 9년만에 최대 실적","아시아투데이 강태윤 기자 = KG모빌리티는 지난달 내수 4043대, 수출 6805대...",003620,1
13001,2023-08-02,"KG모빌리티, 2분기 영업익 188억 흑자전환",[헤럴드경제=증권부] KG모빌리티는 2분기 연결기준 영업이익이 187억8100만원으...,003620,1
13002,2023-08-02,"KG모빌리티, 2Q 영업익 188억 경영 정상화 '속도'",KG모빌리티가 7년 만에 상반기 흑자전환을 이뤄냈다. KG그룹 품에 안긴 이후 경영...,003620,1
13003,2023-08-02,"KG모빌리티, 2분기 영업익 188억 전년비 흑자전환[주목 e공시]",KG모빌리티는 올 2분기 연결 기준 영업이익이 188억원으로 전년 동기 대비 흑자전...,003620,1
13004,2023-08-02,"‘토레스 효과’ KG모빌리티, 7년만에 흑자 전환",[대한경제=강주현 기자]KG모빌리티가 중형 스포츠유틸리티차(SUV) 토레스의 판매 ...,003620,1
13005,2023-08-02,"KG모빌리티, 상반기 영업익 282억원 7년 만에 흑전",역대 상반기 최대 매출 기록\n판매 증가 원가 구조 개선 주효KG모빌리티는 올 2분...,003620,1
13006,2023-08-02,"KG모빌리티, 영업익 282억 7년만에 흑자전환",KG모빌리티가 7년 만에 상반기 흑자 전환에 성공하면서 KG그룹 편입 효과가 빠르게...,003620,1
13007,2023-08-02,"KG모빌리티, 7년 만에 웃었다 '하반기 수출주력'",KG모빌리티가 7년 만에 상반기 흑자를 기록했다. 원가절감과 함께 판매에 주력한 결...,003620,1
13008,2023-08-02,"'토레스 효과' KG모빌리티, 2분기 영업익 188억 2개 분기 연속 흑자",KG모빌리티는 올해 2분기 영업이익 188억원을 기록하며 2개 분기 연속 흑자를 달...,003620,1
13009,2023-08-02,"KG모빌리티, 7년 만에 흑자전환",상반기 매출 2조원 돌파 \n재무구조 개선 목표\nKG모빌리티가 상반기 역대 최대 ...,003620,1


In [ ]:
label_1.iloc[1638]['본문']

'대한통운 제외 2Q 매출 4조4233억 영업익 2358억원부자재 가격에 인건비 등 제반비용 상승으로 CJ제일제당이 상반기 실적이 크게 감소했다.\n \n\nCJ제일제당은 CJ대한통운을 제외한 올해 2분기 매출이 4조4233억원, 영업이익 2358억원을 기록했다고 7일 공시했다. 매출은 전년동기대비 3.7% 감소했지만 영업이익은 무려 40.1%나 줄었다.\n \n..'

In [ ]:
labelling['ticker'].unique()

array(['138930', '097950', '139130', '000210', '375500', '383220',
       '078930', '042670', '204320', '011200', '298050', '175330',
       '105560', '002380', '003620', '030200', '003550', '051900',
       '011070', '066570', '006260', '001120', '035420', '011790',
       '034730', '001740', '402340', '096770', '285130', '017670',
       '000660', '002710', '010130', '024110', '251270', '005250',
       '004370', '003090', '069620', '001440', '026960', '014820',
       '034020', '336260', '023530', '280360', '004000', '004990',
       '005300', '011170', '138040', '002840', '028050', '006400',
       '028260', '029780', '000810', '068270', '055550', '112610',
       '090430', '137310', '450080', '111770', '007310', '001800',
       '316140', '008730', '271940', '030000', '185750', '323410',
       '192820', '021240', '039490', '028670', '022100', '047050',
       '103140', '352820', '161890', '161390', '042700', '128940',
       '009240', '009420', '300720', '000880', '009830', '2722

In [ ]:
labelling.loc[labelling['ticker'] == '069620']['label'].value_counts()

,count
label,
0,4079
1,2560
-1,655


In [ ]:
df_pred_hope['sentiment'].value_counts()

,count
sentiment,
중립,3265
호재(긍정),2785
악재(부정),1244


In [ ]:
df_pred_hope

,일자,제목,본문,ticker,pred_label,prob_neg,prob_neu,prob_pos,sentiment
0,20150101,"대웅제약, “국민의 건강은 우리가” 신약개발에 밤낮 매진",◆ 광복 70돌 다시 뛰는 코리아 / 해방둥이 기업들의 도전 ◆\n 대웅제약은 19...,69620,1,0.0060,0.0029,0.9910,호재(긍정)
1,20150102,"대웅제약, 제약업계 최초 CP 등급평가 ‘AA’ 등급 획득",[이투데이] 강인효 기자(kangih@etoday.co.kr)대웅제약은 공정거래위원...,69620,0,0.0928,0.9053,0.0019,중립
2,20150102,"대웅제약, 2015년 시무식에서 ‘글로벌 제약기업 도약’ 다짐",[이투데이] 강인효 기자(kangih@etoday.co.kr)대웅제약이 글로벌 제약...,69620,0,0.0045,0.9941,0.0015,중립
3,20150104,"대웅제약 ""2015년 글로벌 제약기업 도약""",지속적인 R&D 투자와 글로벌 시장 거점 확대 통한 수출 활성화 목표[아시아경제 이...,69620,0,0.0105,0.9848,0.0047,중립
4,20150105,"대웅제약, CP등급평가 ‘AA’ 최고등급 획득",브레이크뉴스 문흥수 기자= 대웅제약이 공정거래위원회가 주관하는 CP 등급...,69620,0,0.0155,0.9828,0.0018,중립
...,...,...,...,...,...,...,...,...,...
7289,20241223,"대웅제약, 펙수클루 브라질 수출 공급계약 해지 “다른 파트너사 찾는다”",위식도역류질환 치료제 ‘펙수클루’. 대웅제약 \n대웅제약이 브라질 EMS S/A와 ...,69620,1,0.3262,0.3181,0.3557,호재(긍정)
7290,20241223,"대웅제약, 859억 규모 펙스클루 공급계약 해지...""브라질 EMS S/A 계약불이행""",[이데일리 김지완 기자] 대웅제약(069620)이 859억원 규모의 펙스클루 공급계...,69620,-1,0.9940,0.0049,0.0011,악재(부정)
7291,20241223,"대웅제약, 859억 규모 펙스클루 브라질 수출 공급계약 해지",대웅제약은 브라질 EMS S/A와 체결한 859억원 규모 펙수클루(성분명 펙수프라잔...,69620,1,0.2964,0.2574,0.4463,호재(긍정)
7292,20241223,"[공시]대웅제약, 펙수프라잔 브라질 수출 공급 계약 해지",[파이낸셜뉴스] 대웅제약은 지난 2020년 8월 체결한 펙수프라잔(Fexupraza...,69620,-1,0.9830,0.0114,0.0056,악재(부정)


In [ ]:
labelling.loc[labelling['ticker'] == '000720']['label'].value_counts()

,count
label,
0,9147
1,4507
-1,549


In [ ]:
df_pred_hope2 = pd.read_csv('/content/drive/MyDrive/현대건설_predicted_results_hope.csv')
df_pred_hope2

,일자,제목,본문,ticker,pred_label,prob_neg,prob_neu,prob_pos,sentiment
0,20150102,"정수현 현대건설 사장 ""체질개선 기업문화 혁신""","정수현(사진) 현대건설 사장은 2일 신년사를 통해 ""본부와 실 사이에 사업부라는 새...",720,0,0.1286,0.5693,0.3020,중립
1,20150102,"[신년사]정수현 현대건설 사장 ""글로벌 건설 명가로 도약하자""",[정수현 현대건설 사장] \n \n아주경제 강영관 기자 = 정수현 현대건설 사장은 ...,720,0,0.0034,0.9900,0.0066,중립
2,20150102,"[신년사]정수현 현대건설 사장 ""사업체질 개선 기업문화 혁신 이루자""",[아시아경제 김민진 기자] 정수현 현대건설 사장은 2일 시무식에서 사업체질 개선과 ...,720,0,0.0045,0.9926,0.0029,중립
3,20150102,[신년사] 정수현 현대건설 사장 “올해 경영방침은 글로벌 건설리더를 지향하는 새로운...,[이투데이] 구성헌 기자(carlove@etoday.co.kr)“지속적인 체질개선과...,720,-1,0.8266,0.1694,0.0040,악재(부정)
4,20150102,[신년사 전문] 정수현 현대건설 사장,"- 임직원 여러분, 안녕하십니까. \n\n2015년 을미년 새해가 밝았습니다. 먼저...",720,0,0.0205,0.9779,0.0016,중립
...,...,...,...,...,...,...,...,...,...
14198,20231230,"현대건설, 평촌 공작부영 리모델링 수주 5년 연속 정비사업 1위",아시아투데이 전원준 기자 = 현대건설이 경기 평촌 공작부영아파트 리모델링 사업을 따...,720,1,0.0005,0.0008,0.9987,호재(긍정)
14199,20231231,"현대엔지니어링-현대건설, 평촌 공작부영아파트 리모델링 수주",현대엔지니어링 현대건설 컨소시엄이 총 6463억 원 규모의 경기 ‘안양 평촌 공작부...,720,1,0.0007,0.0009,0.9984,호재(긍정)
14200,20231231,"현대엔지-현대건설 컨소, 평촌 공작부영 리모델링 수주","총 1942가구, 6463억원 규모 \n수평 별동 증축 232가구 늘어\n\n\n현...",720,1,0.0005,0.0007,0.9988,호재(긍정)
14201,20231231,"""압구정 現代아파트 신화 이어간다"" 현대건설, 압구정 TF팀 신설",현대건설이 서울 압구정 아파트지구 재건축을 위한 조직을 신설했다. 국내 최대 부촌인...,720,0,0.0349,0.9621,0.0030,중립


In [ ]:
df_pred_hope2['sentiment'].value_counts()

,count
sentiment,
중립,8247
호재(긍정),4832
악재(부정),1124
